In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd

In [35]:
class SolarPanelDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file, delimiter=';')
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        image = Image.open(self.root_dir + img_name).convert('L')  # Convert to grayscale

        if self.transform:
            image = self.transform(image)

        crack_label = self.data.iloc[idx, 1]
        inactive_label = self.data.iloc[idx, 2]

        return image, crack_label, inactive_label

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 10
batch_size = 16
learning_rate = 0.001
random_state = 42
test_size = 0.2
val_size = 0.2

In [38]:
csv_file = '/home/woody/iwso/iwso092h/dl/data.csv'
root_dir = '/home/woody/iwso/iwso092h/dl/'

In [32]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert images to RGB
    transforms.ToTensor(),
])

In [44]:
dataset = SolarPanelDataset(csv_file, root_dir, transform=transform)

total_samples = len(dataset)
test_samples = int(test_size * total_samples)
train_val_samples = total_samples - test_samples

train_val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_val_samples, test_samples],
                                                               generator=torch.Generator().manual_seed(random_state))
train_samples = int((1 - val_size) * train_val_samples)
val_samples = train_val_samples - train_samples
train_dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [train_samples, val_samples],
                                                           generator=torch.Generator().manual_seed(random_state))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create the VGG19 model
num_classes = 2  # Number of classes: crack and no crack
model = models.vgg19(pretrained=True)

/home/hpc/iwso/iwso092h/miniconda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hpc/iwso/iwso092h/miniconda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [47]:
for param in model.parameters():
    param.requires_grad = False

in_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(in_features, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [48]:
for epoch in range(num_epochs):
    model.train()
    for i, (images, crack_labels, _) in enumerate(train_dataloader):
        images = images.to(device)
        crack_labels = crack_labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, crack_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for images, crack_labels, _ in val_dataloader:
            images = images.to(device)
            crack_labels = crack_labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            val_predictions.extend(predicted.cpu().numpy())
            val_labels.extend(crack_labels.cpu().numpy())

    val_f1 = f1_score(val_labels, val_predictions)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Validation F1 Score: {val_f1}")

model.eval()
test_predictions = []
test_labels = []
with torch.no_grad():
    for images, crack_labels, _ in test_dataloader:
        images = images.to(device)
        crack_labels = crack_labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(crack_labels.cpu().numpy())

test_f1 = f1_score(test_labels, test_predictions)
print(f"Test F1 Score: {test_f1}")

KeyboardInterrupt: 